<a href="https://colab.research.google.com/github/parseny/Generative-Models-2024/blob/main/Assignment%203/3_ddpm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Models
***

**Autumn 2024**

## Assignment 3


In this assignment we implement DDPM - Denoising Diffusion Probabilistic Models (2020)

In simple terms, we get an image from data and add noise step by step. Then We train a model to predict that noise at each step and use the model to generate images.

The following definitions and derivations show how this works. For details please refer to the paper https://arxiv.org/abs/2006.11239

## Forward Process

The forward process adds noise to the data $x_0 \sim q(x_0)$, for $T$ timesteps.

\begin{align}
q(x_t | x_{t-1}) = \mathcal{N}\big(x_t; \sqrt{1-  \beta_t} x_{t-1}, \beta_t \mathbf{I}\big) \\
q(x_{1:T} | x_0) = \prod_{t = 1}^{T} q(x_t | x_{t-1})
\end{align}

where $\beta_1, \dots, \beta_T$ is the variance schedule.

We can sample $x_t$ at any timestep $t$ with,

\begin{align}
q(x_t|x_0) &= \mathcal{N} \Big(x_t; \sqrt{\bar\alpha_t} x_0, (1-\bar\alpha_t) \mathbf{I} \Big)
\end{align}

where $\alpha_t = 1 - \beta_t$ and $\bar\alpha_t = \prod_{s=1}^t \alpha_s$

## Reverse Process

The reverse process removes noise starting at $p(x_T) = \mathcal{N}(x_T; \mathbf{0}, \mathbf{I})$
for $T$ time steps.

\begin{align}
p_\theta(x_{t-1} | x_t) &= \mathcal{N}\big(x_{t-1};
 \mu_\theta(x_t, t), \Sigma_\theta(x_t, t)\big) \\
p_\theta(x_{0:T}) &= p_\theta(x_T) \prod_{t = 1}^{T} p_\theta(x_{t-1} | x_t) \\
p_\theta(x_0) &= \int p_\theta(x_{0:T}) dx_{1:T}
\end{align}

$\theta$ are the parameters we train.

## Loss

We optimize the ELBO (from Jenson's inequality) on the negative log likelihood.

\begin{align}
\mathbb{E}[-\log p_\theta(x_0)]
 &\le \mathbb{E}_q [ -\log \frac{p_\theta(x_{0:T})}{q(x_{1:T}|x_0)} ] \\
 &=L
\end{align}

The loss can be rewritten as  follows.

\begin{align}
L
 &= \mathbb{E}_q [ -\log \frac{p_\theta(x_{0:T})}{q(x_{1:T}|x_0)} ] \\
 &= \mathbb{E}_q [ -\log p(x_T) - \sum_{t=1}^T \log \frac{p_\theta(x_{t-1}|x_t)}{q(x_t|x_{t-1})} ] \\
 &= \mathbb{E}_q [
  -\log \frac{p(x_T)}{q(x_T|x_0)}
  -\sum_{t=2}^T \log \frac{p_\theta(x_{t-1}|x_t)}{q(x_{t-1}|x_t,x_0)}
  -\log p_\theta(x_0|x_1)] \\
 &= \mathbb{E}_q [
   D_{KL}(q(x_T|x_0) \Vert p(x_T))
  +\sum_{t=2}^T D_{KL}(q(x_{t-1}|x_t,x_0) \Vert p_\theta(x_{t-1}|x_t))
  -\log p_\theta(x_0|x_1)]
\end{align}

$D_{KL}(q(x_T|x_0) \Vert p(x_T))$ is constant since we keep $\beta_1, \dots, \beta_T$ constant.

### Computing $L_{t-1} = D_{KL}(q(x_{t-1}|x_t,x_0) \Vert p_\theta(x_{t-1}|x_t))$

The forward process posterior conditioned by $x_0$ is,

\begin{align}
q(x_{t-1}|x_t, x_0) &= \mathcal{N} \Big(x_{t-1}; \tilde\mu_t(x_t, x_0), \tilde\beta_t \mathbf{I} \Big) \\
\tilde\mu_t(x_t, x_0) &= \frac{\sqrt{\bar\alpha_{t-1}}\beta_t}{1 - \bar\alpha_t}x_0
                         + \frac{\sqrt{\alpha_t}(1 - \bar\alpha_{t-1})}{1-\bar\alpha_t}x_t \\
\tilde\beta_t &= \frac{1 - \bar\alpha_{t-1}}{1 - \bar\alpha_t} \beta_t
\end{align}

The paper sets $\Sigma_\theta(x_t, t) = \sigma_t^2 \mathbf{I}$ where $\sigma_t^2$ is set to constants
$\beta_t$ or $\tilde\beta_t$.

Then,
$$p_\theta(x_{t-1} | x_t) = \mathcal{N}\big(x_{t-1}; \mu_\theta(x_t, t), \sigma_t^2 \mathbf{I} \big)$$

For given noise $\epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ using $q(x_t|x_0)$

\begin{align}
x_t(x_0, \epsilon) &= \sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon \\
x_0 &= \frac{1}{\sqrt{\bar\alpha_t}} \Big(x_t(x_0, \epsilon) -  \sqrt{1-\bar\alpha_t}\epsilon\Big)
\end{align}

This gives,

\begin{align}
L_{t-1}
 &= D_{KL}(q(x_{t-1}|x_t,x_0) \Vert p_\theta(x_{t-1}|x_t)) \\
 &= \mathbb{E}_q \Bigg[ \frac{1}{2\sigma_t^2}
 \Big \Vert \tilde\mu(x_t, x_0) - \mu_\theta(x_t, t) \Big \Vert^2 \Bigg] \\
 &= \mathbb{E}_{x_0, \epsilon} \Bigg[ \frac{1}{2\sigma_t^2}
  \bigg\Vert \frac{1}{\sqrt{\alpha_t}} \Big(
  x_t(x_0, \epsilon) - \frac{\beta_t}{\sqrt{1 - \bar\alpha_t}} \epsilon
  \Big) - \mu_\theta(x_t(x_0, \epsilon), t) \bigg\Vert^2 \Bigg] \\
\end{align}

Re-parameterizing with a model to predict noise

\begin{align}
\mu_\theta(x_t, t) &= \tilde\mu \bigg(x_t,
  \frac{1}{\sqrt{\bar\alpha_t}} \Big(x_t -
   \sqrt{1-\bar\alpha_t}\epsilon_\theta(x_t, t) \Big) \bigg) \\
  &= \frac{1}{\sqrt{\alpha_t}} \Big(x_t -
  \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t, t) \Big)
\end{align}

where $\epsilon_\theta$ is a learned function that predicts $\epsilon$ given $(x_t, t)$.

This gives,

\begin{align}
L_{t-1}
&= \mathbb{E}_{x_0, \epsilon} \Bigg[ \frac{\beta_t^2}{2\sigma_t^2 \alpha_t (1 - \bar\alpha_t)}
  \Big\Vert
  \epsilon - \epsilon_\theta(\sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon, t)
  \Big\Vert^2 \Bigg]
\end{align}

That is, we are training to predict the noise.

### Simplified loss

$$L_{\text{simple}}(\theta) = \mathbb{E}_{t,x_0, \epsilon} \Bigg[ \bigg\Vert
\epsilon - \epsilon_\theta(\sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon, t)
\bigg\Vert^2 \Bigg]$$

This minimizes $-\log p_\theta(x_0|x_1)$ when $t=1$ and $L_{t-1}$ for $t\gt1$ discarding the
weighting in $L_{t-1}$. Discarding the weights $\frac{\beta_t^2}{2\sigma_t^2 \alpha_t (1 - \bar\alpha_t)}$
increase the weight given to higher $t$ (which have higher noise levels), therefore increasing the sample quality.

This file implements the loss calculation and a basic sampling method that we use to generate images during
training.

$\epsilon_\theta(x_t, t)$ is a UNet model.

In [7]:
from typing import List, Tuple, Optional

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Load file unet.py to the filesystem of colab
from unet import UNet

In [9]:
def gather(consts: torch.Tensor, t: torch.Tensor):
    """Gather consts for t and reshape to feature map shape"""
    c = consts.gather(-1, t)
    return c.reshape(-1, 1, 1, 1)

### Explanations to the code
[1] $\bar\alpha_t = \prod_{s=1}^t \alpha_s$

[2] \begin{align}
        q(x_t|x_0) &= \mathcal{N} \Big(x_t; \sqrt{\bar\alpha_t} x_0, (1-\bar\alpha_t) \mathbf{I} \Big)
        \end{align}

[3] gather $\alpha_t$ and compute $\sqrt{\bar\alpha_t} x_0$

[4] $(1-\bar\alpha_t) \mathbf{I}$

[5] \begin{align}
        q(x_t|x_0) &= \mathcal{N} \Big(x_t; \sqrt{\bar\alpha_t} x_0, (1-\bar\alpha_t) \mathbf{I} \Big)
        \end{align}

[6] $\epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$

[7] \begin{align}
        p_\theta(x_{t-1} | x_t) &= \mathcal{N}\big(x_{t-1};
        \mu_\theta(x_t, t), \sigma_t^2 \mathbf{I} \big) \\
        \mu_\theta(x_t, t)
          &= \frac{1}{\sqrt{\alpha_t}} \Big(x_t -
            \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t, t) \Big)
        \end{align}

[8] gather $\bar\alpha_t$

[9] $\frac{\beta}{\sqrt{1-\bar\alpha_t}}$

[10] $$\frac{1}{\sqrt{\alpha_t}} \Big(x_t -
              \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t, t) \Big)$$

[11] $\epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$

[12] $$L_{\text{simple}}(\theta) = \mathbb{E}_{t,x_0, \epsilon} \Bigg[ \bigg\Vert
        \epsilon - \epsilon_\theta(\sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon, t)
        \bigg\Vert^2 \Bigg]$$

[13] $\epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$

[14] Get $\epsilon_\theta(\sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon, t)$

In [10]:
beta = torch.linspace(0.0001, 0.02, 10)
alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)  # Используем torch.cumprod для кумулятивного произведения
print("Alpha_bar (cumulative product of alpha):", alpha_bar)


gather(alpha_bar, torch.tensor([1,2,3]))

Alpha_bar (cumulative product of alpha): tensor([0.9999, 0.9976, 0.9931, 0.9864, 0.9776, 0.9667, 0.9537, 0.9389, 0.9222,
        0.9037])


tensor([[[[0.9976]]],


        [[[0.9931]]],


        [[[0.9864]]]])

In [14]:
class DenoiseDiffusion:
    """
    ## Denoise Diffusion
    """

    def __init__(self, eps_model: nn.Module, n_steps: int, device: torch.device):
        """
        * eps_model - epsilon_theta(x_t, t) model
        * n_steps - t
        * device - the device to place constants on
        """
        super().__init__()
        self.eps_model = eps_model

        # Create beta_1 ... beta_T linearly increasing variance schedule
        self.beta = torch.linspace(0.0001, 0.02, n_steps).to(device)

        # alpha_t = 1 - beta_t
        self.alpha = 1 - self.beta
        # [1]
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        # T
        self.n_steps = n_steps
        # sigma^2 = beta
        self.sigma2 = self.beta

    def q_xt_x0(self, x0: torch.Tensor, t: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Get q(x_t|x_0) distribution

        [2]
        """
        # [3]
        mean = gather(self.alpha_bar, t) ** 0.5 * x0
        # [4]
        var = 1 - gather(self.alpha_bar, t)
        return mean, var

    def q_sample(self, x0: torch.Tensor, t: torch.Tensor, eps: Optional[torch.Tensor] = None):
        """
        Sample from q(x_t|x_0)

        [5]
        """

        # [6]
        if eps is None:
            eps = torch.randn_like(x0)

        # get q(x_t|x_0)
        mean, var = self.q_xt_x0(x0, t)
        # Sample from q(x_t|x_0)
        return mean + (var ** 0.5) * eps

    def p_sample(self, xt: torch.Tensor, t: torch.Tensor):
        """
        Sample from p_theta(x_{t-1}|x_t)

        [7]
        """

        # epsilon_theta(x_t, t)
        eps_theta = self.eps_model(xt, t)
        print(f"epsilon_theta(x_t, t)\n{eps_theta}")
        # [8]
        alpha_bar = gather(self.alpha_bar, t)
        print(f"alpha_bar\n{alpha_bar}")
        # alpha_t
        alpha = gather(self.alpha, t)
        print(f"alpha\n{alpha}")
        # [9]
        beta = gather(self.beta, t)
        eps_coef = beta / (1 - alpha_bar) ** 0.5
        print(f"eps_coef\n{eps_coef}")
        # [10]
        mean = (1 / alpha_bar ** 0.5) * (xt - eps_coef * eps_theta)
        print(f"mean\n{mean}")

        # sigma^2
        var = gather(self.sigma2, t)

        # [11]
        eps = eps = torch.randn_like(xt)
        print(f"eps\n{eps}")

        # Sample
        return mean + (var ** .5) * eps

    def loss(self, x0: torch.Tensor, noise: Optional[torch.Tensor] = None):
        """
        Simplified Loss

        [12]
        """
        # Get batch size
        batch_size = x0.size(0)
        print(f"batch_size {batch_size}")
        # Get random t for each sample in the batch
        t = torch.randint(0, self.n_steps, (batch_size,), device=x0.device, dtype=torch.long)

        # [13]
        if noise is None:
            noise = torch.randn_like(x0)

        # Sample x_t for q(x_t|x_0)
        xt = self.q_sample(x0, t, noise)
        print(f"xt\n{xt}")
        # [14]
        eps_theta = self.eps_model(xt, t)
        print(f"eps_theta\n{eps_theta}")
        # MSE loss
        return torch.mean((noise - eps_theta) ** 2)

### Explanations to the code
[1] $x_T \sim p(x_T) = \mathcal{N}(x_T; \mathbf{0}, \mathbf{I})$

[2] Sample from $p_\theta(x_{t-1}|x_t)$



In [15]:
class MNISTDataset(torchvision.datasets.MNIST):
    def __init__(self, image_size):
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(image_size),
            torchvision.transforms.ToTensor(),
        ])

        super().__init__("data", train=True, download=True, transform=transform)

    def __getitem__(self, item):
        return super().__getitem__(item)[0]

def plot_samples(tensor):
    # Assuming you have a tensor of size torch.Size([16, 1, 32, 32])
    # Convert the tensor to a numpy array
    images = tensor.numpy()

    # Reshape the images to be of size (16, 32, 32)
    images = np.reshape(images, (16, 32, 32))

    # Create a figure with a grid of subplots
    fig, axes = plt.subplots(nrows=4, ncols=4)

    # Iterate over the images and plot them on the subplots
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(images[i], cmap='gray')
        ax.axis('off')

    # Show the plot
    plt.show()


class Configs:
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # U-Net model for epsilon_theta(x_t, t)
    eps_model: UNet
    # DDPM algorithm
    diffusion: DenoiseDiffusion

    # Number of channels in the image. 3 for RGB.
    image_channels: int = 1
    # Image size
    image_size: int = 32
    # Number of channels in the initial feature map
    n_channels: int = 64
    # The list of channel numbers at each resolution.
    # The number of channels is `channel_multipliers[i] * n_channels`
    channel_multipliers: List[int] = [1, 2, 2, 4]
    # The list of booleans that indicate whether to use attention at each resolution
    is_attention: List[int] = [False, False, False, True]

    # Number of time steps T
    n_steps: int = 1_000
    # Batch size
    batch_size: int = 64
    # Number of samples to generate
    n_samples: int = 16
    # Learning rate
    learning_rate: float = 2e-5

    # Number of training epochs
    epochs: int = 5

    # Dataset
    dataset: torch.utils.data.Dataset = MNISTDataset(image_size)
    # Dataloader
    data_loader: torch.utils.data.DataLoader

    # Adam optimizer
    optimizer: torch.optim.Adam

    def init(self):
        # Create epsilon_theta(x_t, t) model
        self.eps_model = UNet(
            image_channels=self.image_channels,
            n_channels=self.n_channels,
            ch_mults=self.channel_multipliers,
            is_attn=self.is_attention,
        ).to(self.device)

        # Create DDPM class
        self.diffusion = DenoiseDiffusion(
            eps_model=self.eps_model,
            n_steps=self.n_steps,
            device=self.device,
        )

        # Create dataloader
        self.data_loader = torch.utils.data.DataLoader(self.dataset, self.batch_size, shuffle=True, pin_memory=True)
        # Create optimizer
        self.optimizer = torch.optim.Adam(self.eps_model.parameters(), lr=self.learning_rate)

    def sample(self):
        with torch.no_grad():
            # [1]
            x = torch.randn(self.n_samples, self.image_channels, self.image_size, self.image_size, device=self.device)
            print(x)
            # Remove noise for T steps
            progress_bar = tqdm(range(self.n_steps))
            for t_ in progress_bar:
                progress_bar.set_description(f"Sampling")
                # t
                t = self.n_steps - t_ - 1
                # [2]
                x = self.diffusion.p_sample(x, x.new_full((self.n_samples,), t, dtype=torch.long))
            print(x)
            print("-"*50)

            # Log samples
            plot_samples(x.detach().cpu())

    def train(self, epoch):
        # Iterate through the dataset
        progress_bar = tqdm(self.data_loader)
        for data in progress_bar:
            # Increment global step
            progress_bar.set_description(f"Epoch {epoch + 1}")
            # Move data to device
            data = data.to(self.device)

            # Make the gradients zero
            self.optimizer.zero_grad()

            # Calculate loss
            loss = self.diffusion.loss(data)
            # Compute gradients
            loss.backward()

            # Take an optimization step
            self.optimizer.step()

            # Track the loss
            progress_bar.set_postfix(loss=loss.detach().cpu().numpy())

    def run(self):
        for epoch in range(self.epochs):
            # Train the model
            self.train(epoch)
            # Sample some images
            self.sample()


# Create configurations
configs = Configs()

# Initialize
configs.init()

# Start and run the training loop
configs.run()

Epoch 1:   0%|          | 0/938 [00:00<?, ?it/s]

batch_size 64
xt
tensor([[[[-2.2426e-01,  1.4322e+00, -1.5438e-01,  ...,  1.7611e+00,
           -5.2675e-01, -1.3608e+00],
          [-9.5763e-02,  2.7397e-01, -3.5343e-01,  ..., -5.1428e-01,
            4.3963e-02,  1.0126e+00],
          [ 5.7745e-01, -5.2418e-01, -9.4116e-01,  ...,  8.8333e-01,
            1.2845e+00,  6.4045e-01],
          ...,
          [-1.0916e+00,  3.6844e-01, -1.2685e+00,  ...,  9.1202e-01,
           -3.7307e-01, -1.0829e+00],
          [ 3.5043e-01,  1.6573e+00,  1.0553e+00,  ..., -5.6981e-01,
           -5.7773e-01,  6.5816e-02],
          [-4.4103e-01, -5.7662e-01, -1.0651e-02,  ...,  9.9537e-01,
           -1.3923e+00, -1.7097e+00]]],


        [[[ 4.7867e-02,  6.9165e-01, -6.0698e-01,  ...,  3.3133e-01,
            8.1156e-01,  7.8016e-01],
          [-1.4943e-01,  8.1341e-02, -1.8356e-01,  ..., -3.1091e-01,
            2.0309e-01, -2.2664e-03],
          [-3.4213e-01,  3.1049e-01, -8.0175e-01,  ..., -1.2250e+00,
            1.0830e-01,  1.3493e-02],
 

Epoch 1:   0%|          | 1/938 [00:00<08:09,  1.91it/s, loss=1.0394685]

batch_size 64
xt
tensor([[[[ 3.6344e-01,  2.5900e-01, -1.6759e+00,  ..., -4.8209e-01,
            6.7624e-01,  1.0985e+00],
          [ 3.1010e-01,  8.7285e-01,  1.2693e+00,  ...,  6.8192e-02,
            7.5356e-01,  8.3679e-01],
          [ 2.6263e-01, -5.9613e-01, -2.6634e-01,  ...,  1.2717e+00,
            6.5781e-01, -8.4107e-01],
          ...,
          [-2.7947e-01,  6.4195e-01,  6.8330e-01,  ..., -1.5055e+00,
            6.3110e-01, -7.3850e-01],
          [-1.4872e-01, -4.7804e-01, -2.3663e-01,  ..., -5.9556e-01,
           -6.9530e-01,  6.8199e-01],
          [-1.2049e-01,  6.5579e-01, -5.3643e-01,  ...,  1.0683e+00,
            2.1613e+00, -4.3853e-01]]],


        [[[-8.0363e-01,  3.7641e-01, -4.0223e-01,  ...,  2.0537e-01,
           -1.0750e-01,  3.3387e-01],
          [-3.5284e-01, -3.0426e-01, -4.5934e-01,  ...,  1.3248e-01,
            1.0814e-01,  4.9610e-01],
          [-1.5360e-01,  2.0262e-01,  7.6547e-03,  ...,  1.4681e-02,
            4.7041e-01,  1.1862e-01],
 

Epoch 1:   0%|          | 2/938 [00:00<07:18,  2.14it/s, loss=0.9842684]

...

batch_size 64
xt
tensor([[[[ 5.1841e-01,  1.5778e-01,  3.8648e-01,  ..., -1.9706e-01,
           -4.4896e-01, -1.3333e+00],
          [-2.6320e-01, -8.0434e-01,  2.3660e-01,  ...,  2.9630e-01,
           -6.7823e-02,  2.4959e-01],
          [ 1.1615e+00, -3.1143e-01,  3.7137e-01,  ..., -2.0262e-01,
            6.6252e-01, -6.3509e-02],
          ...,
          [ 8.9269e-01, -7.7015e-02, -3.1826e-01,  ..., -4.9585e-01,
            5.4731e-02,  3.3630e-01],
          [ 2.8544e-01,  1.7892e-01,  4.4316e-01,  ..., -7.2038e-01,
            1.3129e-01, -7.3345e-02],
          [ 2.6592e-01, -7.9412e-01,  4.6906e-01,  ..., -5.3999e-02,
            3.0783e-01, -1.1604e-01]]],


        [[[-6.3579e-01, -1.1267e-02, -7.9362e-02,  ..., -1.2321e+00,
            6.5215e-01,  7.8956e-01],
          [-1.1571e+00,  1.2311e+00,  1.0616e+00,  ...,  1.7394e+00,
           -5.7355e-01, -6.8268e-01],
          [-6.2122e-01,  1.2348e+00,  7.4948e-01,  ...,  1.2769e+00,
           -1.1052e-02,  2.4410e-01],
 

Epoch 1:   3%|▎         | 31/938 [00:13<06:42,  2.25it/s, loss=0.22088155]

batch_size 64
xt
tensor([[[[ 0.1184, -0.2421,  0.1584,  ..., -0.1943,  0.1378, -0.3020],
          [-0.1759, -0.0646,  0.1051,  ..., -0.0905,  0.2416,  0.2919],
          [-0.2954,  0.0951, -0.0357,  ..., -0.2872,  0.3034, -0.3042],
          ...,
          [ 0.3011, -0.0043, -0.0923,  ...,  0.3258,  0.4825, -0.6007],
          [-0.2358, -0.1561,  0.1580,  ...,  0.3702,  0.1222, -0.1310],
          [ 0.5020,  0.1695,  0.1549,  ..., -0.2231,  0.2219, -0.8553]]],


        [[[-1.8279, -0.0769, -1.2472,  ..., -0.8862,  0.3655, -0.5269],
          [-1.1606, -2.0846, -0.2761,  ...,  0.5587, -0.9940,  0.7065],
          [-1.0124, -0.9097,  1.2255,  ...,  0.8903, -0.7052, -1.4093],
          ...,
          [ 2.8524,  0.2497, -0.2454,  ..., -0.5188, -1.5399, -0.1621],
          [ 1.0478, -0.2945,  1.1114,  ...,  0.9764,  0.5104, -0.5579],
          [-1.1774, -0.1119,  1.5941,  ...,  0.3562, -0.2551,  0.3147]]],


        [[[-0.3693, -1.5592,  0.8970,  ..., -0.9551, -1.7835, -0.5925],
         

Epoch 1:   3%|▎         | 31/938 [00:13<06:48,  2.22it/s, loss=0.22088155]


KeyboardInterrupt: 